In [ ]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(""), "../../"))
sys.path.insert(0, PROJECT_ROOT)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from postgres import fetch_table,run_query

import logging

logging.basicConfig(level=logging.INFO)  # DEBUG < INFO < WARNING < ERROR < CRITICAL

pd.set_option("display.max_columns", None)  # Show all columns

In [ ]:
# Fetch analytics tables from the mart
tracks_df = fetch_table("mart_all_tracks")
setlist_df = fetch_table("mart_setlist_history")
albums_df = fetch_table("mart_all_albums")
track_setlist_df = fetch_table("mart_track_setlist_similarity_scores")

In [ ]:
# fetch combined table with filtering of albums not relevant and 

query="""
with track_link_cte as (
	select 
 		sim.event_set_song_id
 		,sim.track_id
 		,row_number() over (partition by sim.event_set_song_id order by similarity_score desc nulls last) as similarity_rank_post_filter
	 from 
	 	analytics_mart.mart_track_setlist_similarity_scores as sim 
	 	join analytics_mart.mart_all_tracks as all_tr
	 		on sim.track_id = all_tr.track_id
	 where true
	 	and all_tr.album_id not in ('4bGIhBIGsziCakPziupVmQ','39P5R1B5XF1dGhpojrukVA','06y12naBReESEe7u2OFG7j')
)
, track_link_filtered_cte as (
	select 
 		*
	 from 
	 	track_link_cte
	 where true
	 	and similarity_rank_post_filter = 1
)
select 
	msh.*
	,tr.*
from 
	analytics_mart.mart_setlist_history as msh
	left join track_link_filtered_cte as tl_cte
		on msh.event_set_song_id = tl_cte.event_set_song_id
	left join analytics_mart.mart_all_tracks as tr
		on tl_cte.track_id = tr.track_id
where true
"""

df = run_query(query)

df.head(5)

In [ ]:
print(df['event_date'].dtype)

In [ ]:
df.event_id.nunique()

In [ ]:
df.event_date.min()

In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')

df['year'] = df['event_date'].dt.year

event_counts = df.groupby('year')['event_url'].nunique()

event_counts

In [ ]:
event_counts.plot(kind='bar', title='Unique Events per Year', figsize=(10, 4))

In [ ]:
df['song_name'].value_counts()

In [ ]:
df_recent = df[df.event_date >= '2019-01-01'].reset_index(drop=True)

df_recent

In [ ]:
df_recent['covid_era'] = df_recent['event_date'].apply(
    lambda d: 'pre' if d < pd.Timestamp('2020-03-01') else 'post'
)

song_counts = (
    df_recent
    .groupby(['covid_era', 'song_name'])
    .size()
    .reset_index(name='count')
)

# Normalize by total shows per era (since they likely did fewer shows post-2020)
total_shows = df_recent.groupby('covid_era')['event_date'].nunique().to_dict()

song_counts['per_show'] = song_counts.apply(
    lambda row: row['count'] / total_shows[row['covid_era']], axis=1
)

In [ ]:
# Pivot table to see differences
pivot = song_counts.pivot(index='song_name', columns='covid_era', values='per_show').fillna(0)

# Add a column for difference
pivot['change'] = pivot['post'] - pivot['pre']

# See most increased/decreased songs
pivot.sort_values('change', ascending=False).head(10)  # Most more-played post-COVID
pivot.sort_values('change').head(10)                   # Most dropped post-COVID

In [ ]:
pivot.loc[pivot['change'].abs() > 0.1][['pre', 'post']].plot(kind='barh', figsize=(10, 12))

In [ ]:
query_song_prob="""
with track_link_cte as (
	select 
 		sim.event_set_song_id
 		,sim.track_id
 		,row_number() over (partition by sim.event_set_song_id order by similarity_score desc nulls last) as similarity_rank_post_filter
	 from 
	 	analytics_mart.mart_track_setlist_similarity_scores as sim 
	 	join analytics_mart.mart_all_tracks as all_tr
	 		on sim.track_id = all_tr.track_id
	 where true
	 	and all_tr.album_id not in ('4bGIhBIGsziCakPziupVmQ','39P5R1B5XF1dGhpojrukVA','06y12naBReESEe7u2OFG7j')
)
, track_link_filtered_cte as (
	select 
 		*
	 from 
	 	track_link_cte
	 where true
	 	and similarity_rank_post_filter = 1
)
, setlist_clean_cte as (
	select 
		msh.*
		,tr.*
	from 
		analytics_mart.mart_setlist_history as msh
		left join track_link_filtered_cte as tl_cte
			on msh.event_set_song_id = tl_cte.event_set_song_id
		left join analytics_mart.mart_all_tracks as tr
			on tl_cte.track_id = tr.track_id
	where true
)
, events_per_tour as (
	select 
		event_tour_id
		,count(distinct event_id) as total_events
	from 
		setlist_clean_cte 
	group by 
		1
)
select 
	st.event_tour_id
	,st.song_name
    ,st.event_tour
	,count(distinct st.event_id) as total_times_played
	,max(tour.total_events) as total_chances_to_play
	,(count(distinct st.event_id)*1.00) / max(tour.total_events) as song_probability_in_tour
from 
	setlist_clean_cte as st
	join events_per_tour as tour
		on st.event_tour_id = tour.event_tour_id
where true 
group by 
	1,2,3
;
"""

song_tour_prob_df = run_query(query_song_prob)

song_tour_prob_df.head(5)

In [ ]:
from scipy.stats import entropy


song_tour_prob_df['norm_prob'] = song_tour_prob_df.groupby('event_tour_id')['song_probability_in_tour'].transform(lambda x: x / x.sum())

# Define a helper function
def calculate_entropy(group):
    probs = group['norm_prob'].values
    return pd.Series({'entropy': entropy(probs, base=2)})

entropy_by_tour = song_tour_prob_df.groupby('event_tour_id').apply(calculate_entropy).reset_index()

entropy_by_tour.head(30)

In [ ]:
import numpy as np

entropy_shows = (
    song_tour_prob_df
    .merge(entropy_by_tour, on="event_tour_id")
    .groupby(['event_tour_id', 'event_tour'])[['total_chances_to_play', 'entropy']]
    .max()
    .reset_index()
)


plt.figure(figsize=(10, 6))
plt.scatter(entropy_shows['total_chances_to_play'], entropy_shows['entropy'])

np.random.seed(42)

for _, row in entropy_shows.iterrows():
    y_offset = np.random.choice([-0.1, 0.1])
    if row['entropy'] > 4.5 or row['entropy'] < 4.5:  # label outliers
        plt.text(
            row['total_chances_to_play'] + 0.5
            , row['entropy'] + y_offset
            , row['event_tour']
            , fontsize=6)

plt.xlabel("Number of Shows in Tour")
plt.ylabel("Setlist Entropy (bits)")
plt.title("Setlist Entropy vs. Tour Length (mewithoutYou)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
tours_df = (
    df.groupby('event_tour_id')['event_date']
    .min()
    .reset_index()
    )

tours_df = tours_df.merge(entropy_shows, on='event_tour_id')

tours_df['effective_setlist_size']= 2 ** tours_df['entropy']

tours_df_sorted = tours_df.sort_values('event_date').reset_index(drop=True)

plt.figure(figsize=(14, 6))
plt.bar(tours_df_sorted['event_tour'], tours_df_sorted['entropy'])

plt.xticks(rotation=75, ha='right', fontsize=8)
plt.ylabel("Setlist Entropy (bits)")
plt.title("Setlist Entropy by Tour (Chronological Order)")
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.bar(tours_df_sorted['event_tour'], tours_df_sorted['effective_setlist_size'])

plt.xticks(rotation=75, ha='right', fontsize=8)
plt.ylabel("Effective Setlist Size (2^entropy)")
plt.title("Estimated Setlist Diversity by Tour (mewithoutYou)")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()